# Scrap data from der Spiegel

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urljoin
import newspaper
from newspaper import Article

#### 1. Get list of articles available

In [2]:
spiegel_topics = ['http://spiegel.de/politik/',
                  'http://spiegel.de/thema/meinung/',
                  'http://spiegel.de/wirtschaft/',
                  'http://spiegel.de/panorama/',
                  'http://spiegel.de/sport/',
                  'http://spiegel.de/kultur/',
                  'http://spiegel.de/netzwelt/',
                  'http://spiegel.de/wissenschaft/',
                  'http://spiegel.de/gesundheit/']
spiegel_url = 'http://spiegel.de'
topics_de = ['politik', 'meinung', 'wirtschaft', 'panorama', 'sport', 'kultur', 'netzwelt', 'wissenschaft', 'gesundheit']
topics_en = ['politics', 'opinion', 'economy', 'society', 'sport', 'culture', 'technology', 'science', 'health']
website = 'http://spiegel.de/schlagzeilen'

In [3]:
def get_list_articles(website):
    r = requests.get(website)
    soup = BeautifulSoup((r.content), "lxml")

    news_collection = []
    div = soup.find("div", {"class": "schlagzeilen-content schlagzeilen-overview"})
    for a in div.find_all('a', title=True):
        news = {}
        news['title'] = a.text
        news['topic'] = a.find_next_sibling('span').text
        news['url'] = urljoin(website, a.get('href'))
        news_collection.append(news)    
    return news_collection    

In [4]:
news_collection = get_list_articles(website)

In [5]:
news_collection = pd.DataFrame.from_records(news_collection)

In [6]:
# add new fields
news_collection['keywords'] = ''
news_collection['meta_description'] = ''
news_collection['text'] = ''

In [7]:
news_collection.head(2)

,title,topic,url,keywords,meta_description,text
0,Höhere Bußgelder: Bundesländer fordern abschre...,"(Auto, 10:33)",http://spiegel.de/auto/aktuell/bussgeld-raser-...,,,
1,"Kanye Wests Twitter-Philosophie: ""Sei einfach ...","(Panorama, 10:19)",http://spiegel.de/panorama/leute/kanye-wests-t...,,,


#### 2. Get content of articles

In [8]:
def scrap_text_for_each_article(Series):
    a = Article(Series['url'], language='de')
    a.download()
    a.parse()
    #import pdb; pdb.set_trace()
    Series['text'] = a.text
    Series['keywords'] = a.keywords
    Series['meta_description'] = a.meta_description
    return Series

In [9]:
news_complete = news_collection.apply(lambda x: scrap_text_for_each_article(x), axis=1)

In [10]:
news_complete.head()

,title,topic,url,keywords,meta_description,text
0,Höhere Bußgelder: Bundesländer fordern abschre...,"(Auto, 10:33)",http://spiegel.de/auto/aktuell/bussgeld-raser-...,[],Die Bundesländer sprechen sich nach einem Medi...,Die Bundesländer setzen sich für höhere Bußgel...
1,"Kanye Wests Twitter-Philosophie: ""Sei einfach ...","(Panorama, 10:19)",http://spiegel.de/panorama/leute/kanye-wests-t...,[],Kanye West übt sich wieder in Twitterei. Nach ...,Ob ihm der öffentliche Trubel im Kardashian-Cl...
2,Stasi am Telefon: Die Tonspur der Schnüffelei,"(einestages, 10:14)",http://spiegel.de/einestages/stasi-hoerspiel-d...,[],"Klicken, Knacken, Knistern und alle Facetten d...","""Teilnehmer!"" Schon das allererste Wort des Ge..."
3,Im Auto: Dreijährige schießt in USA auf schwan...,"(Panorama, 09:05)",http://spiegel.de/panorama/justiz/usa-dreijaeh...,[],Die Waffe lag geladen auf der Mittelkonsole: A...,In den USA hat ein dreijähriges Mädchen seine ...
4,Umstrittene neue Funktion: Was Facebooks Gesic...,"(Netzwelt, 09:01)",http://spiegel.de/netzwelt/netzpolitik/faceboo...,[],In den kommenden Wochen bietet Facebook seine ...,Facebook will Nutzern ein wenig mehr Kontrolle...


#### 3. Get comments from those articles

In [11]:
def get_comments_from_articles(list_articles):
    comments = []
    for article in list_articles:
        soup = BeautifulSoup(requests.get(article).content)
        article_comments = []
        for comm in soup.select("div.article-comment-title"):
            try:
                article_comments.append(comm.a.text.strip()+comm.find_next("div", "js-article-post-full-text").text.strip())
            except:
                print("{} article comments not available.".format(article))
        article_comments = ', '.join(article_comments)
        comments.append(article_comments)
    return comments    

In [12]:
comments = get_comments_from_articles(news_complete.url.tolist())

/Users/celsadiaz/.pyenv/versions/3.6.2/envs/wepublic/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/celsadiaz/.pyenv/versions/3.6.2/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://spiegel.de/einestages/stasi-hoerspiel-die-tonspur-der-schnueffelei-a-1202363.html article comments not available.
http://spiegel.de/einestages/street-photography-aus-new-york-wie-carrie-boretz-ihre-stadt-sieht-a-1187940.html article comments not available.
http://spiegel.de/einestages/street-photography-aus-new-york-wie-carrie-boretz-ihre-stadt-sieht-a-1187940.html article comments not available.
http://spiegel.de/einestages/street-photography-aus-new-york-wie-carrie-boretz-ihre-stadt-sieht-a-1187940.html article comments not available.
http://spiegel.de/einestages/street-photography-aus-new-york-wie-carrie-boretz-ihre-stadt-sieht-a-1187940.html article comments not available.


In [13]:
news_complete['comments'] = comments

In [14]:
news_complete.to_pickle('../../data/news_complete_19_04_19.pkl')